### Todo:
- Fix scrapper exception bug
- Loop through users if fail

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import subprocess
import names
from selenium.webdriver.common.by import By
import math
import random
import pandas as pd
import os
import json
import re
import sys
import shutil
from datetime import datetime

### Constants

In [2]:
email = "maskradiogreece@gmail.com"
password = "code12345"

In [3]:
def convert_str_to_number(x):
    number = 0
    num_map = {'K':1000, 'M':1000000, 'B':1000000000}
    if x.isdigit():
        number = int(x)
    else:
        if len(x) > 1:
            number = float(x[:-1]) * num_map.get(x[-1].upper(), 1)
    return int(number)

In [4]:
def clear_unlabeled_images(folder_name):
    regex = re.compile(r'^.[^\.].')
    
    files = []
    for file in os.listdir(f"./{folder_name}"):
        files.append(file)

    files_to_clear = list(filter(regex.search, files))
    for file in files_to_clear:
        os.remove(f"./{folder_name}/{file}")

In [5]:
def scrap_profile(name,count):
    try:
        command = "instagram-scraper --user " + name + " -u " + email + " -p " + password + " -m " + str(count) + " --cookiejar cookies" 
        subprocess.check_output(command)
    except subprocess.CalledProcessError as e:
        raise Exception("Sorry, papala")

In [6]:
def evaluate_profile(folder_name):
    file = open(f'./{folder_name}/{folder_name}.json', encoding="utf8")
    data = json.load(file)
    pattern = re.compile(r'\d+_\d+_\d+_n.jpg')
    images = []
    likes = []
    for item in data["GraphImages"]:
        finds = re.search(pattern,item["display_url"])
        # If the post is at least 2 days old then add it to data or else skip it
        if (datetime.today() - datetime.fromtimestamp(item["taken_at_timestamp"])).days > 2:
            images.append(finds[0])
            likes.append(item["edge_media_preview_like"]["count"])
    maxLikes = max(likes)
    if maxLikes < 500:
        file.close()
        print("DELETING ====> Not Enough Likes")
        shutil.rmtree(rf"./{folder_name}/") #removes the folder.
        return False
    for index,name in enumerate(images):
        try:
            os.rename(rf"./{folder_name}/{name}",rf"./{folder_name}/{round(likes[index]/maxLikes,2):.2f} {folder_name}{name}")
        except:
            pass
    file.close()
    time.sleep(2)
    # Clear Images with no score
    print("STEP 3 ====> Clearing unecessary images")
    clear_unlabeled_images(folder_name)
    # Todo resize all images in folder  
    print("STEP 4 ====> Moving folder")
    shutil.move(f"./{folder_name}/",f"./Images/{folder_name}/")
    return True

### Open driver and Login to Instagram

In [7]:
driver = webdriver.Chrome("./chromedriver_win32/chromedriver.exe")
driver.maximize_window()
driver.get("https://www.instagram.com/")
time.sleep(5)
buttons = driver.find_elements_by_tag_name("button")
time.sleep(1)
buttons[2].click()
login_inputs = driver.find_elements_by_tag_name("input")
time.sleep(1)
login_inputs[0].send_keys(email)
time.sleep(1)
login_inputs[1].send_keys(password)
time.sleep(1)
buttons[0].click()
time.sleep(5)
notification_button = driver.find_elements(By.XPATH, '//button[text()="Όχι τώρα"]')
time.sleep(1)
notification_button[0].click()

# Scraping Code

In [ ]:
num_images = 0
throttle = 600
while True:
    driver.get("https://www.instagram.com/")
    inputs = driver.find_elements_by_tag_name("input")
    # The 3rd one is the search 
    search = inputs[2]
    random_female_name = names.get_first_name(gender='female') 
    search.send_keys(random_female_name)
    time.sleep(4)
    # Get List of Users
    user_list = driver.find_elements(By.XPATH, '//div[@role="none"]')
    # Create List 
    profile_links = []
    for user in user_list:
        link = user.find_element_by_tag_name("a")
        link = link.get_attribute("href")
        if link.find("explore") == -1:
            profile_links.append(link)
    searched_links = pd.read_csv("./Links.csv")
    searched_links = searched_links["Links"].to_list()
    loop = 1 
    for link in profile_links:
        name = link.split("/")[3]
        if link in searched_links:
            print(f"SKIPPING ====> USER {name} ALREADY SCRAPED ")
            continue
        driver.get(link);
        time.sleep(5)
        try:
            is_private = driver.find_element(By.XPATH, "//*[text()[contains(., 'Αυτός ο λογαριασμός είναι ιδιωτικός')]]" )
            continue 
        except: 
            post = driver.find_element(By.XPATH, "//*[text()[contains(., 'δημοσιεύσεις') or contains(., 'δημοσίευση')]]" )
            num_posts = int(post.find_element_by_tag_name("span").get_attribute("innerHTML").replace(",",""))
            if num_posts < 20:
                print("SKIPPING ====> LESS THAN 10 POSTS")
                continue
            followers = driver.find_element(By.XPATH, "//*[text()[contains(., 'ακόλουθοι')]]" )
            num_followers = convert_str_to_number(followers.find_element_by_tag_name("span").get_attribute("innerHTML").replace(",",""))
            if num_followers < 3000:
                print("SKIPPING ====> LESS THAN 3000 FOLLOWERS")
                continue
            if loop % 5 == 0:
                # Every 5 loop wait between 3-5 min
                wait_time = random.randint(180,300)

            else: 
                # Wait between 2-3 min
                wait_time = random.randint(120,180)
            loop += 1
            print(f"STEP 0 ====> Waiting {wait_time} seconds")
            time.sleep(wait_time)
            get_num_posts = math.floor(num_posts * 0.5)
            get_num_posts = min(math.floor(num_posts * 0.5),200)
            print(f"STEP 1 ====> Getting {get_num_posts} from {name}")
            try:
                scrap_profile(name,get_num_posts)
                print("STEP 2 ====> Evaluating images")
                added = evaluate_profile(name)
                if added:
                    num_images += get_num_posts
                    print(f"SUM ==============> {num_images} in total")
            except Exception as e:
                print("ERROR : "+str(e))
                print(f"There was error on getting {name}")
                # Sleep bettwen throttle to throttle + 2 minutes
                # We get throttling from API
                sleep_time = random.randint(throttle, throttle + 60)
                print(f"Going to sleep for {sleep_time}s")
                time.sleep(sleep_time)
                # Increase throttle by 1 min
                throttle += 60
                # Append Failed Link to end of array to try and fetch again 
                profile_links.append(link)
                continue
            searched_links.append(link)
            df = pd.DataFrame(searched_links,columns=["Links"])
            df.to_csv('./Links.csv',index=False)

STEP 0 ====> Waiting 121 seconds
STEP 1 ====> Getting 24 from dorothy
STEP 2 ====> Evaluating images
STEP 3 ====> Clearing unecessary images
STEP 4 ====> Moving folder
SUM ==============> 24 in total
STEP 0 ====> Waiting 123 seconds
